In [ ]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random
from urllib.error import URLError
from urllib.error import HTTPError


In [ ]:
import ssl
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [ ]:
def CheckLinkError(url): 
    try:
        response = urlopen(url, context=ssl_context)
        html_data = response.read().decode('utf-8')
        return 0
    except URLError as e:
            return 1

In [ ]:
def getTopicLink(bs, domain): 
    TopicLink = []
    DivTopic = bs.find('ol', class_= "discussionListItems")
    for link in DivTopic.find_all('a', href= re.compile('^(thread)((?!page).)*$')): 
        if link.attrs['href'] is not None:
            newlink = domain + '/' +link.attrs['href']
            checkLink = CheckLinkError(newlink)
            if checkLink == 0: 
                if newlink not in TopicLink:
                    TopicLink.append(newlink)
    return TopicLink

In [ ]:
def removeLinkerror(lstAccountLinks): 
    linkerror = []
    for i in range(len(lstAccountLinks)): 
        url = lstAccountLinks[i]
        try:
            response = urlopen(url, context=ssl_context)
            html_data = response.read().decode('utf-8')
        except URLError as e:
            linkerror.append(i)
    result = lstAccountLinks
    for index in sorted(linkerror, reverse=True):
        del result[index]
    return result

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
TopicLink = set()
for i in tqdm(range(10000)): 
    url = 'https://f319.com/forums/thi-truong-chung-khoan.3/page-' + str(i)
    try: 
        html = urlopen(url, context=ssl_context)
    except Exception: 
        pass 
    bs = BeautifulSoup(html, 'html.parser')
    domain = '{}://{}'.format(urlparse(url).scheme, urlparse(url).netloc)
    newTopic = getTopicLink(bs, domain)
    TopicLink.update(newTopic)


In [ ]:
TopicLink = list(TopicLink)
TopicLink = removeLinkerror(TopicLink)

In [ ]:
import pandas as pd
dataLink = pd.DataFrame()
dataLink['Link'] = TopicLink
dataLink.to_csv('TopicLink.csv')